In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import GEOparse as Geo
import seaborn as sns
from pandas.core.ops import comp_method_OBJECT_ARRAY
from collections import Counter

os.chdir('../../')
os.getcwd()

'/home/max/mcrc-cetuximab-analysis'

In [2]:
fpkms_df = pd.read_csv("raw/GSE183984_ASAN_RNASEQ_FPKM_ensg.csv", index_col=0).T
raw_counts_df = pd.read_csv("raw/GSE183984_ASAN_RNASEQ_raw_counts_ensg.csv", index_col=0).T
data_ensemble_genes = list(fpkms_df.columns)

tpms_df = fpkms_df.T.div(fpkms_df.sum(axis=1)).T * 10 ** 6

In [3]:
def parse_gtf(gtf_path):
    ensemble_gene_id_to_length = Counter()
    ensemble_gene_id_to_version = dict()
    ensemble_gene_id_to_hgnc = dict()
    ensemble_gene_id_to_biotype = dict()
    with open(gtf_path, "r") as f:
        for li, line in enumerate(f):
            # print(line)
            if line.startswith("#"):
                continue
            
            parts = line.strip().split()
            chromosome = parts[0]
            entry_source_0 = parts[1]
            entry_type = parts[2]
            start = int(parts[3])
            end = int(parts[4])
            info = dict()
            for i in range(8, len(parts) - 1, 2):
                key = parts[i]
                val = parts[i + 1][1:-2]
                if key == 'gene_name' and val == 'havana':
                    print(key, val, i)
                info[key] = val
            
            if entry_type == 'gene' and 'gene_id' in info and 'gene_name' in info:
                ensemble_gene_id_to_hgnc[info['gene_id']] = info['gene_name']
                if 'gene_biotype' in info:
                    ensemble_gene_id_to_biotype[info['gene_id']] = info['gene_biotype']
            elif entry_type == 'exon':
                version = info['gene_version']
                ensemble_gene_id = info['gene_id']
                ensemble_gene_id_to_version[ensemble_gene_id] = version
                ensemble_gene_id_to_length[ensemble_gene_id] += end - start + 1
                    
    return ensemble_gene_id_to_hgnc, ensemble_gene_id_to_length, ensemble_gene_id_to_version, ensemble_gene_id_to_biotype

ensemble_gene_id_to_hgnc, ensemble_gene_id_to_length, \
 ensemble_gene_id_to_version, ensemble_gene_id_to_biotype = parse_gtf('data/Homo_sapiens.GRCh38.113.gtf')

In [4]:
print('ENS genes with known HGNC name: ', len(ensemble_gene_id_to_hgnc.keys()))
print('ENS genes in raw data: ', len(data_ensemble_genes))
print('ENS genes with known HGNC that are not in raw data: ', len(set(ensemble_gene_id_to_hgnc.keys()) - set(data_ensemble_genes)))
print('ENS genes in raw data for which we don\'t know HGNC: ', len(set(data_ensemble_genes) - set(ensemble_gene_id_to_hgnc.keys())))

ensemble_genes_common = list(set(data_ensemble_genes).intersection(ensemble_gene_id_to_hgnc.keys()))
print('ENS genes from data with known HGNC: ', len(ensemble_genes_common))

ENS genes with known HGNC name:  42745
ENS genes in raw data:  58735
ENS genes with known HGNC that are not in raw data:  597
ENS genes in raw data for which we don't know HGNC:  16587
ENS genes from data with known HGNC:  42148


In [5]:
from collections import Counter

cnt = Counter()
hgnc_to_ensemble_set = dict()
for key in ensemble_gene_id_to_hgnc:
# for key in ensemble_genes_common:
    hgnc = ensemble_gene_id_to_hgnc[key]
    cnt[hgnc] += 1
    if cnt[hgnc] == 1:
        hgnc_to_ensemble_set[hgnc] = {key}
    else:
        hgnc_to_ensemble_set[hgnc].add(key)


In [6]:
def process_columns(df, shrink_to_bg=False):
    print('Initial df cols: ', len(df.columns))
    df = df.drop(columns=list(set(df.columns) - set(ensemble_gene_id_to_hgnc.keys())))
    print('After dropping cols not annotated with HGNC name in GTF: ', len(df.columns))
    
    renamer = dict()
    for egene in ensemble_gene_id_to_hgnc:
        if ensemble_gene_id_to_biotype[egene] == 'protein_coding':
            renamer[egene] = ensemble_gene_id_to_hgnc[egene]

    df = df.rename(columns=renamer)
    df = df.drop(columns=[col for col in df.columns if col.startswith('ENSG')])
    
    print('After renaming to HGNC (and deleting non-protein coding egenes): ', len(df.columns))

    if shrink_to_bg:
        df = df[list(common_hgncs.intersection(df.columns))]
        print('After shrinking to BG HGNCs that are in the table: ', len(df.columns))
    
    mask = df.columns.duplicated()
    for i in range(len(mask)):
        if mask[i]:
            j = 0
            while df.columns[j] != df.columns[i]:
                j += 1
            df[df.columns[j]] += df[df.columns[i]]

    df = df.loc[:,~df.columns.duplicated()]

    print('After summing columns with the same HGNC name: ', len(df.columns))

    return df

raw_counts_df = process_columns(raw_counts_df)
fpkms_df = process_columns(fpkms_df)
tpms_df = process_columns(tpms_df)

Initial df cols:  58735
After dropping cols not annotated with HGNC name in GTF:  42148
After renaming to HGNC (and deleting non-protein coding egenes):  19394
After summing columns with the same HGNC name:  19388
Initial df cols:  58735
After dropping cols not annotated with HGNC name in GTF:  42148
After renaming to HGNC (and deleting non-protein coding egenes):  19394
After summing columns with the same HGNC name:  19388
Initial df cols:  58735
After dropping cols not annotated with HGNC name in GTF:  42148
After renaming to HGNC (and deleting non-protein coding egenes):  19394
After summing columns with the same HGNC name:  19388


In [7]:
import pandas as pd

def extend_id(id):
    fp, sp = id.split('_')
    while len(sp) < 4:
        sp = '0' + sp
    return fp + '_' + sp

# Load annotation
ann = pd.read_csv("data/ann_maxim.csv").set_index('sample_id')

tpms_df.index = tpms_df.index.map(extend_id)
raw_counts_df.index = raw_counts_df.index.map(extend_id)

# Leave only samples from annotation
tpms_df = tpms_df.loc[ann.index]
raw_counts_df = raw_counts_df.loc[ann.index]

log_tpms_df = tpms_df.apply(lambda x: np.log2(1 + x))

In [11]:
log_tpms_df.to_csv('data/log_tpms_from_fpkm_hgnc_filtered_by_ann_maxim.csv')
raw_counts_df.to_csv('data/raw_counts_hgnc_filtered_by_ann_maxim.csv')

In [12]:
our_hgncs = set(log_tpms_df.columns)
bg_hgncs = set(open('data/gene_lists/gnames.txt').read().strip().split())

print('Excess genes compared to BG list: ', len(our_hgncs - bg_hgncs))
print('Missing genes from BG list: ', len(bg_hgncs - our_hgncs))

common_hgncs = our_hgncs.intersection(bg_hgncs)
print('Common genes: ', len(common_hgncs))

for key in hgnc_to_ensemble_set:
    if len(hgnc_to_ensemble_set[key]) > 1 and key is not None:
        print(key)
        print([(egene, ensemble_gene_id_to_length[egene], ensemble_gene_id_to_biotype[egene], ensemble_gene_id_to_version[egene]) for egene in hgnc_to_ensemble_set[key]])

Excess genes compared to BG list:  1444
Missing genes from BG list:  2118
Common genes:  17944
SNORA70
[('ENSG00000253027', 151, 'snoRNA', '1'), ('ENSG00000206958', 136, 'snoRNA', '1'), ('ENSG00000206886', 133, 'snoRNA', '1'), ('ENSG00000200237', 141, 'snoRNA', '1'), ('ENSG00000276094', 95, 'snoRNA', '1'), ('ENSG00000202379', 134, 'snoRNA', '1'), ('ENSG00000252526', 90, 'snoRNA', '1'), ('ENSG00000207098', 135, 'snoRNA', '1'), ('ENSG00000207244', 134, 'snoRNA', '1'), ('ENSG00000251893', 153, 'snoRNA', '2'), ('ENSG00000252505', 98, 'snoRNA', '1'), ('ENSG00000201376', 135, 'snoRNA', '1'), ('ENSG00000251925', 138, 'snoRNA', '1'), ('ENSG00000202389', 135, 'snoRNA', '1'), ('ENSG00000252199', 95, 'snoRNA', '1'), ('ENSG00000278010', 95, 'snoRNA', '1'), ('ENSG00000252258', 133, 'snoRNA', '1'), ('ENSG00000206661', 134, 'snoRNA', '1'), ('ENSG00000252014', 110, 'snoRNA', '1'), ('ENSG00000252657', 126, 'snoRNA', '1'), ('ENSG00000207165', 135, 'snoRNA', '1'), ('ENSG00000251796', 138, 'snoRNA', '1'),